In [118]:
#Moisture evolution (5 minutes)
import numpy as np
import pandas as pd
import random
from datetime import date
from datetime import datetime
from datetime import timedelta
from eto import ETo #For daily evapotranspiration calculation

    
def Kcgenerator(Kcini,Kcmed,Kcend,dini,dcob,dmed,dend):
    global daystot
    global Kc
    #Kc array creation
    daystot=dini+dcob+dmed+dend
    time = np.arange(daystot)
    inivec = np.zeros(dini) 
    inivec = inivec + Kcini
    cobvec=np.arange(Kcini, Kcmed, (Kcmed-Kcini)/dcob)
    medvec = np.zeros(dmed) 
    medvec = medvec + Kcmed
    endvec=np.arange(Kcmed, Kcend, -(Kcmed-Kcend)/dend)
    Kc=np.concatenate([inivec,cobvec,medvec,endvec])

def timevecgenerator(initial_date):
    global timevec
    global time5min
    #global initial_date
    timevec=[]
    time5min = np.arange(daystot*24*12)#Point every 5min
    for i in range(0, len(time5min)):
        initial_date = initial_date + timedelta(minutes=5)
        timevec.append(initial_date)

def KsKigenerator():
    global Ks
    global Ki
    #Ks calculations
    PR=int((FC+PWP)/2)
    PGAP=int(abs(100*(PR-PWP)))
    Kss1=np.ones(int(100*PWP-10))*0.1
    Kss2=np.arange(0.1,1,0.9/(PGAP+10))
    Kss2=Kss2**2
    Kss3=np.ones(100-100*PR)
    Ks=np.concatenate([Kss1,Kss2,Kss3])
    
    FClimit=int(FC*100)
    ni2=100*(SAT-FC) #20
    Ki1=np.ones(FClimit) #
    #Ki2=np.arange(1, 50, 1/6)
    Ki4=np.arange(1,ni2,1)
    Ki4=Ki4*0.5
    Ki4=Ki4**2
    Ki4=Ki4+0.75
    #Ki2=np.array([2,10,30,40,50,64])
    #Ki2=Ki2**IF
    Ki3=np.ones(100-int(SAT*100))*7
    Ki=np.concatenate([Ki1,Ki4,Ki3]) #We add ones just in case


def samplingvecgenerator(SI):
    global samplingvec
    samplingvec=[]
    interval = round((SI/5)-1)
    i=0
    while (i < len(time5min)):
        samplingvec.append(1)
        i += 1
        for j in range(interval):
            samplingvec.append(0)
            i += 1
            
def tdmin(delayacumtime):
    days, hours, minutes = delayacumtime.days, delayacumtime.seconds // 3600, delayacumtime.seconds // 60 % 60
    minutestot=days*24*60+hours*60+minutes
    return minutestot

def crop_init(crop):
    Kcgenerator(crop["Kcini"], crop["Kcmed"], crop["Kcend"],crop["dini"],crop["dgrow"],crop["dmed"],crop["dend"])
    #Kc(Kcini,Kcmed,Kcend,dini,dcob,dmed,dend)
    
def ET0generator(weather,initial_date_in):
    final_date= initial_date_in + timedelta(days=daystot+1)
    global ET
    global Rainvec
    global Rainvec_5min
    Rainvec = [] 
    Rainvec_5min = []
    et1 = ETo()
    inidate = initial_date_in.strftime("%Y-%m-%d")
    endate = final_date.strftime("%Y-%m-%d")
    #inidate=datetime.strptime(inidate,"%Y-%m-%d")
    #endate=datetime.strptime(endate,"%Y-%m-%d")
    et1.param_est(weather[inidate:endate], freq, z_msl, lat, lon, TZ_lon, z_u, K_rs=0.16, a_s=0.25, b_s=0.5, alb=0.23)
    #et1.param_est(weather['2010-04-01':'2010-04-30'], freq, z_msl, lat, lon, TZ_lon, z_u, K_rs=0.16, a_s=0.25, b_s=0.5, alb=0.23)
#___Rain vector generator__________________________________________________________________________
    ET=et1.eto_fao(max_ETo=15, min_ETo=0, interp=True, maxgap=15)
    if(crop['name']!='lettuce'):#Only for outside crops.
        Rainvec=weather.loc[inidate:endate,"PRECIPITACION"]
        for i in range (0,daystot):
            if (float(Rainvec[i])>0):
                for j in range (3*12):
                    Rainvec_5min.append(float(Rainvec[i])/(3*12))
                for k in range (21*12):
                    Rainvec_5min.append(0)
            else:
                for l in range (24*12):
                    Rainvec_5min.append(0)
            

def simulation():
    global readingvec
    global policyquality
    global wcvec
    global delay_wc_total
    global delay_min_total
    global irrigationnumber
    global wcQ
    global timeQ
    global Year_stress
    global VoIvec
    global VoI_AHP_vec
    VoIvec = []
    VoI_AHP_vec = []
    wc=initial_wc
    delay_wc_total=0 #Reset accumulated wc delay
    irrigationnumber=0
    delay_min_total=initial_date-initial_date#IMPROVE, Reset accumulated time delay
    readingvec = []
    wcvec=[]
    Total_stress=[]
    Year_stress=0
    irrigationflag=1
    opmoment=0
    last_measure=timevec[0]
    Age=0
    stages_vec=crop['stages_vec']
    stagestot=crop['stages']
    #readingvec.append(-1)#to solve not concordance problem
    for i in range(0, len(time5min)):
        wcindex=int(round((wc)*100)-1)#Avoids extra calculations
        dateindex=int(round(i/(288))-1)#Obtains the day, avoids calculations
        if(crop['name']!='lettuce'):
            wc=wc-(ET[dateindex]/11520)*Kc[dateindex]*Ki[wcindex]*Ks[wcindex]+Rainvec_5min[i]/100
        else:
            wc=wc-(ET[dateindex]/11520)*Kc[dateindex]*Ki[wcindex]*Ks[wcindex]
        wcvec.append(wc)#Stote the value of WC
        if wc>SAT:#To avoid unreal values of soil moisture
            wc=SAT
       
        if (wc <= wcthreshold):
            wc=IT #Watering until target
    


#____________________________________Parameters____________________________________________________________________
lettuce = {'name':'lettuce','plant_date': datetime(2010, 4, 1, 00, 00, 00),
  'dini': 20, 'dgrow': 30,'dmed':15,'dend':10, 'Kcini': 0.75 , 'Kcmed': 1.35 , 'Kcend': 1.2, 'name_map':'color_map_lettuce.csv',
  'file_name':'almeria10-19.csv','msl': 21,'stages': 1,'stages_vec':[1],
  'SAT':0.47, 'FC': 0.40,'PWP':0.30,'IT':0.46,'wcthreshold':0.23}
corn = {'name':'corn','plant_date': datetime(2010, 4, 1, 00, 00, 00),
  'dini': 30, 'dgrow': 40,'dmed':50,'dend':30, 'Kcini': 0.35 , 'Kcmed': 1.2 , 'Kcend': 0.35, 'name_map':'color_map_corn.csv',
  'file_name':'Huesca10-19.csv','msl': 488,'stages': 10,'stages_vec':[0.5,0.5,0.55,0.65,0.8,0.8,0.4,0.3,0.2,0.15],
  'SAT':0.48, 'FC': 0.35,'PWP':0.20,'IT':0.38,'wcthreshold':0.24}
vineyard = {'name':'vineyard','plant_date': datetime(2011, 3, 15, 00, 00, 00),
  'dini': 30, 'dgrow': 60,'dmed':40,'dend':80, 'Kcini': 0.3 , 'Kcmed': 0.7 , 'Kcend': 0.45,'name_map':'color_map_vineyard.csv',
  'file_name':'Tomelloso11-19.csv','msl': 660,'stages': 8,'stages_vec':[0.35,0.35,0.45,0.6,0.75,0.75,0.25,0.25],
  'SAT':0.48, 'FC': 0.20,'PWP':0.13,'IT':0.25,'wcthreshold':0.13}

#_________General Parameters___________
crop=lettuce #Set type of crop
initial_wc = crop['FC'] #Initial water content
ET0 = 0.1 #Daily nominal EvapoTranspiration in VWC% (ex: hibiscus=0.1)
ET0=ET0/(24*12)
SI = 60#sampling interval in minutes, must be multiple of 5
MaxSI = 60

#____SOIL PARAMETERS-VWC(0-1)__________
SAT = crop['SAT'] #Saturation level
FC = crop['FC'] #Field capacity
PWP = crop['PWP'] #Permanent wilting point
wcthreshold = crop['wcthreshold'] #Irrigation threshold
IT = crop['IT'] #Irrigation target

#_______Parameters for ETo_____________
z_msl = crop["msl"] #Elevation of the met station above mean sea level (m) (only needed if P is not in df).
z_u = 2
lat = 39.14 #Latitud
lon = -2.94 #Longitud
TZ_lon = -2.94 # The longitude of the center of the time zone (dec deg) (only needed if calculating ETo hourly)
freq = 'D' #D for daily calculations

#____________________________________MAIN________________________________________________________________

Kcgenerator(crop["Kcini"], crop["Kcmed"], crop["Kcend"],crop["dini"],crop["dgrow"],crop["dmed"],crop["dend"])
initial_date = crop["plant_date"] #Full format, includes hour
timevecgenerator(initial_date)
KsKigenerator()
weather_=pd.read_csv(crop["file_name"])
weather_['datetime'] = pd.to_datetime(weather_['datetime'], format="%Y-%m-%d")
weather=weather_.set_index('datetime', drop = False)
ET0generator(weather,initial_date)#Generates 5min ETo and rainvec(day)


simulation()
wcvec=np.array(wcvec) * 100
wcthreshold=wcthreshold*100
plotterSoil(timevec,wcvec,wcthreshold)






Loading BokehJS ...

In [93]:
print(crop['name'])

{'name': {'name': {'plant_date': datetime.datetime(2010, 4, 1, 0, 0), 'dini': 20, 'dgrow': 30, 'dmed': 15, 'dend': 10, 'Kcini': 0.45, 'Kcmed': 1, 'Kcend': 0.9, 'name_map': 'color_map_lettuce.csv', 'file_name': 'almeria10-19.csv', 'msl': 21, 'stages': 1, 'stages_vec': [1], 'SAT': 0.47, 'FC': 0.43, 'PWP': 0.35, 'IT': 0.46, 'wcthreshold': 0.38}, 'plant_date': datetime.datetime(2010, 4, 1, 0, 0), 'dini': 20, 'dgrow': 30, 'dmed': 15, 'dend': 10, 'Kcini': 0.45, 'Kcmed': 1, 'Kcend': 0.9, 'name_map': 'color_map_lettuce.csv', 'file_name': 'almeria10-19.csv', 'msl': 21, 'stages': 1, 'stages_vec': [1], 'SAT': 0.47, 'FC': 0.43, 'PWP': 0.35, 'IT': 0.46, 'wcthreshold': 0.38}, 'plant_date': datetime.datetime(2010, 4, 1, 0, 0), 'dini': 20, 'dgrow': 30, 'dmed': 15, 'dend': 10, 'Kcini': 0.45, 'Kcmed': 1, 'Kcend': 0.9, 'name_map': 'color_map_lettuce.csv', 'file_name': 'almeria10-19.csv', 'msl': 21, 'stages': 1, 'stages_vec': [1], 'SAT': 0.47, 'FC': 0.43, 'PWP': 0.35, 'IT': 0.46, 'wcthreshold': 0.38}


In [107]:
def plotterSoil(x,y,wcthreshold):
    import numpy as np
    from bokeh.io import output_file, output_notebook
    from bokeh.plotting import figure, show
    from bokeh.models import LinearAxis, Range1d
    from bokeh.embed import file_html
    output_notebook()
    fig = figure(background_fill_color='white',
                 border_fill_color='white',
                 x_axis_label='Date',
                 x_axis_type='datetime',
                 x_axis_location='below',
                 #x_range=(0, 10),
                 y_axis_label='VWC(%)',
                 y_axis_type='linear',
                 y_axis_location='left',
                 y_range=(0, 119),
                 title='Example, Almería',
                 title_location='right',
                 toolbar_location='below',
                 tools=['save', 'undo', 'box_zoom'],
                 plot_width=1200,
                 plot_height=700)
    fig.xaxis.axis_label_text_font_size = "16pt"
    fig.yaxis.axis_label_text_font_size = "16pt"
    fig.title.text_font_size = "16pt"
    fig.xaxis.axis_label_text_font_style = 'normal'
    fig.yaxis.axis_label_text_font_style = 'normal'
    fig.line(x=x, y=wcthreshold,color='red', line_width=1, legend_label="Irrigation threshold (%)")
    fig.line(x=x, y=y,color='green', line_width=1, legend_label="VWC(%)")
    fig.legend.label_text_font_size = "16pt"
    fig.yaxis.major_label_text_font_size = "14pt"
    fig.xaxis.major_label_text_font_size = "14pt"
    show(fig)
    
def plotterSoilBig(x,y,wcthreshold):
    import numpy as np
    from bokeh.io import output_file, output_notebook
    from bokeh.plotting import figure, show
    from bokeh.models import LinearAxis, Range1d
    from bokeh.embed import file_html
    output_notebook()
    fig = figure(background_fill_color='white',
                 border_fill_color='white',
                 x_axis_label='Date',
                 x_axis_type='datetime',
                 x_axis_location='below',
                 #x_range=(0, 10),
                 y_axis_label='VWC(%)',
                 y_axis_type='linear',
                 y_axis_location='left',
                 y_range=(0, 119),
                 title='Example, Almería',
                 title_location='right',
                 toolbar_location='below',
                 tools=['save', 'undo', 'box_zoom'],
                 plot_width=2400,
                 plot_height=2000)
    fig.xaxis.axis_label_text_font_size = "32pt"
    fig.yaxis.axis_label_text_font_size = "32pt"
    fig.title.text_font_size = "32pt"
    fig.xaxis.axis_label_text_font_style = 'normal'
    fig.yaxis.axis_label_text_font_style = 'normal'
    fig.line(x=x, y=wcthreshold,color='red', line_width=2, legend_label="Irrigation threshold (%)")
    fig.line(x=x, y=y,color='green', line_width=2, legend_label="VWC(%)")
    fig.legend.label_text_font_size = "32pt"
    fig.yaxis.major_label_text_font_size = "28pt"
    fig.xaxis.major_label_text_font_size = "28pt"
    show(fig)
    
#plotterSoil(timevec,wcvec,wcthreshold)
